In [29]:
import pandas as pd

from pathlib import Path

In [30]:
types = {
    0: "Bass",
    1: "Brass",
    2: "Keys",
    3: "Lead",
    4: "Organ",
    5: "Pad",
    6: "Percussion",
    7: "Sequence",
    8: "SFX",
    9: "Strings"
}

In [31]:
clean_dir = Path("../patches/clean")

In [32]:
patches = []

for clean_file in clean_dir.glob("*.csv"):

    patch = pd.read_csv(clean_file)
    patch["bank_name"] = clean_file.stem

    patch = patch[[
        "bank_name",
        "patch_name",
        "patch_type"
    ]]

    patch = patch[patch["patch_name"] != "Init"]

    patches.append(patch)

In [33]:
patches = pd.concat(patches)
patches = patches.reset_index(drop=True)

In [34]:
patches["patch_type"] = patches["patch_type"].map(types)

In [35]:
patches[patches.bank_name == "factory_4"]

,bank_name,patch_name,patch_type
1865,factory_4,Ballast,Sequence
1866,factory_4,Digi Express,Bass
1867,factory_4,Black Forest,Sequence
1868,factory_4,O,SFX
1869,factory_4,Future Bits,Keys
...,...,...,...
1924,factory_4,Tremendous Way,Keys
1925,factory_4,Mod Cello,Strings
1926,factory_4,Gritty Volts,Lead
1927,factory_4,Sampler Dirt,Keys


In [36]:
patches.patch_type.value_counts()

patch_type
Pad           698
Bass          682
Sequence      273
Keys          207
Lead          156
SFX           122
Percussion    102
Strings        29
Brass          19
Organ          16
Name: count, dtype: int64